In [ ]:
# Coco Bambu - Transcrição de JSON para SQL

# Instalar bibliotecas necessárias
!pip install -q pandas sqlalchemy

import pandas as pd
import json
from sqlalchemy import create_engine
from IPython.display import display
from google.colab import files

# Upload do arquivo JSON
uploaded = files.upload()

# Carregar o conteúdo do arquivo JSON
filename = list(uploaded.keys())[0]
with open(filename, "r", encoding="utf-8") as f:
    data = json.load(f)

# Função para normalizar dados aninhados
def normalize_json(data):
    location_data = {
        'locRef': data['locRef'],
        'currentUTC': data['curUTC']
    }

    checks, taxes, detail_lines, menu_items = [], [], [], []

    for guest in data['guestChecks']:
        # Fallback entre 'taxation' e 'taxes'
        impostos = guest.get("taxation") or guest.get("taxes")

        check_data = {
            'guestCheckId': guest['guestCheckId'],
            'chkNum': guest['chkNum'],
            'opnBusDt': guest['opnBusDt'],
            'opnUTC': guest['opnUTC'],
            'opnLcl': guest['opnLcl'],
            'clsdBusDt': guest['clsdBusDt'],
            'clsdUTC': guest['clsdUTC'],
            'clsdLcl': guest['clsdLcl'],
            'lastTransUTC': guest['lastTransUTC'],
            'lastTransLcl': guest['lastTransLcl'],
            'lastUpdatedUTC': guest['lastUpdatedUTC'],
            'lastUpdatedLcl': guest['lastUpdatedLcl'],
            'clsdFlag': guest['clsdFlag'],
            'gstCnt': guest['gstCnt'],
            'subTtl': guest['subTtl'],
            'nonTxblSlsTtl': guest['nonTxblSlsTtl'],
            'chkTtl': guest['chkTtl'],
            'dscTtl': guest['dscTtl'],
            'payTtl': guest['payTtl'],
            'balDueTtl': guest['balDueTtl'],
            'rvcNum': guest['rvcNum'],
            'otNum': guest['otNum'],
            'ocNum': guest['ocNum'],
            'tblNum': guest['tblNum'],
            'tblName': guest['tblName'],
            'empNum': guest['empNum'],
            'numSrvcRd': guest['numSrvcRd'],
            'numChkPrntd': guest['numChkPrntd'],
            'locRef': data['locRef']
        }
        checks.append(check_data)

        for tax in impostos or []:
            taxes.append({
                'guestCheckId': guest['guestCheckId'],
                'taxNum': tax['taxNum'],
                'txblSlsTtl': tax['txblSlsTtl'],
                'taxCollTtl': tax['taxCollTtl'],
                'taxRate': tax['taxRate'],
                'type': tax.get('type')
            })

        for detail in guest.get('detailLines', []):
            detail_lines.append({
                'guestCheckLineItemId': detail['guestCheckLineItemId'],
                'guestCheckId': guest['guestCheckId'],
                'rvcNum': detail['rvcNum'],
                'dtlOtNum': detail['dtlOtNum'],
                'dtlOcNum': detail['dtlOcNum'],
                'lineNum': detail['lineNum'],
                'dtlId': detail['dtlId'],
                'detailUTC': detail['detailUTC'],
                'detailLcl': detail['detailLcl'],
                'lastUpdateUTC': detail['lastUpdateUTC'],
                'lastUpdateLcl': detail['lastUpdateLcl'],
                'busDt': detail['busDt'],
                'wsNum': detail['wsNum'],
                'dspTtl': detail['dspTtl'],
                'dspQty': detail['dspQty'],
                'aggTtl': detail['aggTtl'],
                'aggQty': detail['aggQty'],
                'chkEmpId': detail['chkEmpId'],
                'chkEmpNum': detail['chkEmpNum'],
                'svcRndNum': detail['svcRndNum'],
                'seatNum': detail['seatNum'],
                'miNum': detail.get('menuItem', {}).get('miNum')
            })

            if 'menuItem' in detail:
                menu_items.append(detail['menuItem'])

    return {
        'location': pd.DataFrame([location_data]),
        'checks': pd.DataFrame(checks),
        'taxes': pd.DataFrame(taxes),
        'detail_lines': pd.DataFrame(detail_lines),
        'menu_items': pd.DataFrame(menu_items).drop_duplicates()
    }

# Normalizar e criar banco em memória
normalized_data = normalize_json(data)
engine = create_engine('sqlite:///:memory:')
for table_name, df in normalized_data.items():
    df.to_sql(table_name, engine, index=False, if_exists='replace')

# Função para consultas SQL
def run_query(query):
    with engine.connect() as conn:
        result = pd.read_sql(query, conn)
        display(result)

# Executar consultas
print("Banco carregado com sucesso!\n")

print("Tabelas disponíveis:")
run_query("SELECT name FROM sqlite_master WHERE type='table';")

print("Tabela checks:")
run_query("SELECT guestCheckId, chkNum, chkTtl, tblName FROM checks;")

print("Tabela taxes:")
run_query("SELECT * FROM taxes;")

print("Tabela detail_lines:")
run_query("SELECT guestCheckId, guestCheckLineItemId, miNum, dspQty, dspTtl FROM detail_lines;")

print("Tabela menu_items:")
run_query("SELECT * FROM menu_items;")

print("Resumo de comandas:")
run_query("""
SELECT
    c.guestCheckId, c.chkNum, c.tblName, c.empNum, c.chkTtl,
    SUM(d.dspTtl * d.dspQty) AS total_itens,
    COUNT(d.guestCheckLineItemId) AS qtd_itens,
    t.taxRate, t.taxCollTtl
FROM
    checks c
LEFT JOIN detail_lines d ON c.guestCheckId = d.guestCheckId
LEFT JOIN taxes t ON c.guestCheckId = t.guestCheckId
GROUP BY c.guestCheckId
""")

print("Itens mais vendidos:")
run_query("""
SELECT
    m.miNum,
    COUNT(d.guestCheckLineItemId) AS total_vendido,
    SUM(d.dspQty) AS total_quantidade,
    SUM(d.dspTtl * d.dspQty) AS total_valor
FROM
    menu_items m
JOIN detail_lines d ON m.miNum = d.miNum
GROUP BY m.miNum
ORDER BY total_quantidade DESC
""")
